# Resampling

One of the more complex topics when it comes to working with earth-observing satellite data is geographic projections and resampling data to these different projections.

This is how the book "Map Projections" by Battersby describe map projections:

> Map projection is the process of transforming angular (spherical / elliptical) coordinates into planar coordinates. All map projections introduce distortion (e.g., to areas, angles, distances) in the resulting planar coordinates. Understanding what, where, and how much distortion is introduced is an important consideration for spatial computations and visual interpretation of spatial patterns, as well as for general aesthetics of any map.

<img src="http://gistbok.ucgis.org/sites/default/files/figure2-projections.png" width="450px"/>

<sub>Credit: Battersby, S. (2017). Map Projections. The Geographic Information Science & Technology Body of Knowledge (2nd Quarter 2017 Edition), John P. Wilson (ed.). DOI: <a href="http://gistbok.ucgis.org/bok-topics/map-projections">10.22224/gistbok/2017.2.7</a></sub>

To simplify projections and resampling for this tutorial, we can think of resampling as mapping data from one projection to another. Projections describe a flat version of our round Earth that is easier to describe. Different people or visualization tools may prefer a certain projection for the data they look at. When comparing data from multiple sources it can be convenient to have them all on the same projection. Let's explore this with real data.

In [ ]:
%run ../init_notebook.py
from satpy import Scene
from glob import glob

filenames = glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_conus/*.nc')
scn = Scene(reader='abi_l1b', filenames=filenames)
scn.load(['C06'])

In [ ]:
scn['C06'].attrs['area']

The `'area'` attribute of our Satpy data is a special `AreaDefinition` and it defines the geographic area that our data covers. Under `Projection` we see a python dictionary of projection parameters to define that flat plane representation of the Earth. Our ABI data is on a `'geos'` or Geostationary Satellite View coordinate system where position is measured in meters on the the X and Y axes. You can learn more about that on the PROJ site [here](https://proj.org/operations/projections/geos.html). Depending on how we change these parameters we can end up with something like the below view of the Earth or something completely different.

<img src="https://proj.org/_images/geos.png" width="300"/>

Alternatively, we could read data on a completely different projection like the [Lambert Conformal Conic](https://proj.org/operations/projections/lcc.html) projection and be looking at a view like the image below.

<img src="https://proj.org/_images/lcc.png" width="300"/>

What projection we have depends on the satellite and who provided our data to us. What projection we want on the output depends on what our end goal is. Do we want to compare our data with another satellite instrument's? Do we want to view our data in a projection that is less distorted for our region of interest?

In addition to the geostationary projection information, our `AreaDefinition` specifies an exact location on that projection space: the lower-left corner (in projection meters), the upper-right corner, and the number of row and column pixels.

We've already loaded `'C06'`, let's load the `'C05'` channel too and compare them to get a better experience with projections and comparing data.

In [ ]:
scn.load(['C05'])
scn['C05'].attrs['area']

In [ ]:
scn['C06'].attrs['area']

Notice the difference in size (rows and columns) between the two area definitions, the small difference in extents, but also how the projection parameters are exactly the same. This is because these two channels are on the same projection (coordinate system), but their individual pixels are different **resolutions**. Each of channel 5's pixels represents 1 kilometer on the geostationary projection and 2 kilometers for each of channel 6's pixels.

Trying to compare these with normal array operations would be difficult due to the differences in array shape. Instead we can manipulate and resample the data to allow simpler calculations going forward. We can do this using Satpy's `Scene.resample` method which provides multiple algorithms for resampling data. We'll start by using the very simple `'native'` resampler.

In [ ]:
new_scn = scn.resample(resampler='native')
new_scn['C05'].shape == new_scn['C06'].shape

The resample method has given a new `Scene` object with every DataArray we had before, but resampled to the same area or region. By default, it used the highest resolution `AreaDefinition` of the input data (`scn.max_area()`). In this case that's the 1km area from `C05`. If we look at the area of `C06` now we can see it is also at 1km.

In [ ]:
new_scn['C06'].attrs['area']

The `native` resampler we used has two possible operations:

1. If remapping data to a higher resolution, replicate each pixel to make the shape matches.
2. If remapping data to a lower resolution, average/aggregate the pixels to make the shapes match.

Now that our two loaded channels have the same shape, we can easily compare them. Let's take the difference between the two channels and plot it. Note that we are using the resampled `new_scn` and **NOT** the original `scn` object.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

diff = new_scn['C06'] - new_scn['C05']
diff.plot.imshow(vmin=-20, vmax=20)

The `native` resampler can be very useful when you need to combine bands from the same instrument that have different resolutions. However, it is limited because it only operates on data on the same projection and with data that can be easily replicated or aggregated (1km -> 2km but not 375m -> 1km).

To do more complicated resampling we rely on other resampling algorithms, the simplest being the `nearest` resampler for nearest neighbor resampling. Let's create our own custom AreaDefinition to resample to with our own projection. For this, we'll use Pyresample's `create_area_def` utility function. Running the following cell (with the `?`) will open a new pane with the documentation for the `create_area_def` function. We can use this information to start building a new area to resample to. Feel free to close the pane when you're done using it.

In [ ]:
from pyresample import create_area_def
create_area_def?

In [ ]:
my_area = create_area_def('my_area', {'proj': 'lcc', 'lon_0': -95, 'lat_0': 25, 'lat_1': 35},
                          width=1000, height=1000,
                          area_extent=[-105, 20, -90, 40], units='degrees')
my_area

In [ ]:
new_scn = scn.resample(my_area)

We can see from the above AreaDefinition print out that we've created an area using the [Lambert Conformal Conic](https://proj.org/operations/projections/lcc.html) projection that is 1000 rows by 1000 columns. The "Area extent" tells us, in the projection space metered coordinates, where our lower left and upper right corners are. We can look at some commmon area properties to get more information on this area we've created, like the resolution of each pixel.

In [ ]:
my_area.pixel_size_x, my_area.pixel_size_y

So with a 1000x1000 grid of pixels covering this geographic area, we've made it so each pixel represents 1440 meters in the X dimension and 2157 meters in the Y dimension.

Let's use Xarray's plotting utilities again to see what this looks like. We'll specify `vmin` and `vmax` to be between 0% and 100% as good initial limits for the colorbar given the traditional limits of reflectance data.

In [ ]:
plt.figure()
new_scn['C05'].plot.imshow(vmin=0, vmax=100)

So in a couple lines of code we've changed the projection, resolution, and overall size of our data. More importantly, our data that started out at different resolutions have been resampled to the same geographic area and resolution so they can be worked with more easily.

## Dynamic Areas

However, the area we made required us to know the exact region of the Earth we wanted. What if we only knew some of the information and wanted to use our data to fill in the rest. The `create_area_def` function handles this too by creating a `DynamicAreaDefinition` if needed. Let's make a dynamic area where we know the projection and the resolution we want for each pixel (5000 meters). Pay attention to how long each step takes to compute compared to the previous calls.

In [ ]:
my_dynamic_area = create_area_def('my_area', {'proj': 'lcc', 'lon_0': -95, 'lat_0': 25, 'lat_1': 35},
                                  resolution=5000)
my_dynamic_area

In [ ]:
dynamic_scn = scn.resample(my_dynamic_area)
dynamic_scn['C05'].shape

In [ ]:
plt.figure()
dynamic_scn['C05'].plot.imshow(vmin=0, vmax=100)

What you should have noticed is that the call to `resample` took longer than before. This was caused by the calculations necessary to determine how large the resuling area had to be to encompass all of the data.

Additionally, the plotting calls should take longer because we are asking for much more data to be resampled and for more data to be plotted. Remember, the dask arrays only load and compute data when it is needed; in this case the area computation and the plotting.

### Exercise

**Time: 10 minutes**

Use the below cells to plot the ABI data on different projections, tweak the projection parameters, or use dynamic areas to resample all of the area. Note how the output changes as projection parameters and area extents change. Projection changes may not be easy to notice on a small scale.

Some projection suggestions:

1. `{'proj': 'merc', 'lon_0': -97.0}`
2. `{'proj': 'lcc', 'lon_0': -95, 'lat_0': 45, 'lat_1': 55, 'lat_2': 65}`
3. `{'proj': 'stere', 'lon_0': -105.0, 'lat_ts': 25}`

Be careful to not make areas that are too large (many pixels) or you may be waiting a while for processing to finish.

If time permits, try loading other channels to the `scn` object (we currently have C05 and C06), changing the extent of the area, trying other matplotlib features, changing the vmin/vmax parameters, or researching other projections. For a full list of supported projections and their options see the [PROJ documentation](https://proj.org/operations/projections/index.html).

In [ ]:
custom_area = create_area_def('my_area', {'proj': 'merc', 'lon_0': -97.0},
                              width=1000, height=1000,
                              area_extent=[-115, 15, -90, 40], units='degrees')
custom_area

In [ ]:
custom_scn = scn.resample(custom_area)

In [ ]:
plt.figure()
custom_scn['C05'].plot.imshow(vmin=0, vmax=100)

## Swaths of Polar-orbiter data

So far we've been looking at data from the ABI instrument which is onboard a geostationary satellite. It is common for these type of data to be on a fixed grid or area where all of the pixels are equally spaced. Now we will look at a Polar-orbiter's data; the NOAA-20 (JPSS-01) satellite's VIIRS instrument.

In [ ]:
polar_scn = Scene(reader='viirs_sdr', filenames=glob('../data/viirs_sdr/20180511_texas_fire_viirs_sdr/*j01*.h5'))
polar_scn.available_dataset_names()

In [ ]:
polar_scn.load(['I04'])
polar_scn['I04']

There are some important differences we can see in the above `I04` band's information compared to the ABI bands we've looked at so far. First, the five `I`, or Imagery resolution, bands on VIIRS have a spatial resolution of about 375m per pixel. The ABI instrument's highest resolution is 500m per pixel and only one channel (C02) is at that resolution, the others at 1km and 2km resolution. The other 16 `M`, or Moderate resolution, bands on VIIRS are 750m resolution. This is one of the major benefits of most polar-orbiting satellites; more bands at higher resolutions. In fact, the VIIRS data provided for this tutorial was limited because of the large size of the datasets.

Another key thing to notice about polar-orbiting data is that the `'area'` associated with the data is a new type of object called a `SwathDefinition`. A `SwathDefinition` is a simple container holding on to longitude and latitude arrays. This type of geolocation definition is needed when the pixel spacing of the data is non-uniform and cannot be described by a single "grid" of pixels.

In [ ]:
type(polar_scn['I04'].attrs['area'])

In [ ]:
lons, lats = polar_scn['I04'].attrs['area'].get_lonlats()
lons

Let's plot this `I04` band to see what it looks like without any resampling.

In [ ]:
plt.figure()
polar_scn['I04'].plot.imshow()

The edges of this plot look a little strange. This effect is caused by the scanning pattern of the instrument and is known as the "bow-tie effect" where pixels from consecutive scans overlap on the edges of the swath. In the case of VIIRS it is also known as "bow-tie deletion" because the duplicate pixels from this overlap are actually removed from the data. You may also notice that the plot is flipped horizontally (west coast of Mexico is on the right of the image); a side effect of how the data is observed and how we've plotted it.

The most common way to correct these types of effects is to resample the data. Let's reuse the `my_area` definition we created before for ABI, but use it here for VIIRS.

In [ ]:
my_area

In [ ]:
new_polar_scn = polar_scn.resample(my_area)

In [ ]:
plt.figure()
new_polar_scn['I04'].plot.imshow()

This plot shows one of the downsides of typical polar-orbiter data. Although we have many more bands at a higher resolution, they do not cover a single area very often. The fire in Texas we've been looking at had a majority of its activity and growth in about 4 hours. This VIIRS data from NOAA-20, one of two VIIRS instruments in orbit, was the only pass of the instrument that captured any part of the fire. The ABI instrument however took new images every minute.

## Compare ABI and VIIRS

Now that we know we can get ABI and VIIRS data on the area, let's do some comparisons. First, we have to load channels or bands that are similar for both instruments. We'll look at the C05 (1.61µm) and C07 (3.90µm) channels from ABI and the I03 (1.61µm) and I04 (3.74µm) channels from VIIRS. The C05 and I03 bands are both reflectance bands (%) and C07 and I04 are brightness temperature bands (K).

We should also load data from a similar point in time. Given the temporal resolution of VIIRS data, we'll have to select ABI data from the same time (~20:30Z) as our available VIIRS data. Instead of using the CONUS sector of data we used previously, we'll instead switch to using the smaller Mesoscale sector.

We'll recreate all the Scenes and areas to be used for clarity.

In [ ]:
abi_scn = Scene(reader='abi_l1b', filenames=glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_meso/OR_ABI-L1b-RadM1-M3C??_G16_s20181312030*.nc'))
abi_scn.load(['C05', 'C07'])

In [ ]:
viirs_scn = Scene(reader='viirs_sdr', filenames=glob('../data/viirs_sdr/20180511_texas_fire_viirs_sdr/*j01*t203*.h5'))
viirs_scn.load(['I03', 'I04'])

Let's compare the times and size of each band:

In [ ]:
print("ABI: ", abi_scn['C05'].attrs['start_time'])
print(abi_scn['C05'].shape)
print(abi_scn['C07'].shape)
print("VIIRS: ", viirs_scn['I03'].attrs['start_time'])
print(viirs_scn['I03'].shape)
print(viirs_scn['I04'].shape)

We can see that the two ABI bands have a lower resolution (less pixels) and were observed about 8 seconds after the VIIRS data. Since the ABI data is already gridded to the geostationary (`geos`) projection, let's resample the VIIRS bands to the ABI's area. We'll use the special `max_area` method of the ABI `Scene` to get the highest resolution area of the two loaded channels. We'll also need to resample the ABI channels to be at the same resolution and can use the simple `native` resampler like we did before. Remember that by default the `resample` method will use the `max_area()` of the current Scene.

We're also going to use the `dask.persist` function to compute our delayed dask operations up to this point and hold them in memory. This will save us from recomputing these operations in the future, but will use up some of our machine's memory. Normally this isn't needed, but since we might be displaying the data multiple times it would be best to save us from reprocessing it every time. We'll add dask's `ProgressBar` again to get an idea how long each computation is taking.

In [ ]:
import dask
from dask.diagnostics import ProgressBar

with ProgressBar():
    abi_max_scn = abi_scn.resample(resampler='native')
    abi_max_scn['C05'], abi_max_scn['C07'] = dask.persist(abi_max_scn['C05'], abi_max_scn['C07'])

    viirs_max_scn = viirs_scn.resample(abi_scn.max_area())
    viirs_max_scn['I03'], viirs_max_scn['I04'] = dask.persist(viirs_max_scn['I03'], viirs_max_scn['I04'])

print(abi_max_scn['C05'].shape)
print(viirs_max_scn['I03'].shape)

We can see from the printed shapes that our ABI and VIIRS data are the same shape. Now let's plot the 1.61µm band of each Scene (`C05` and `I03`).

In [ ]:
plt.figure()
abi_max_scn['C05'].plot.imshow(vmin=0, vmax=100)

In [ ]:
plt.figure()
viirs_max_scn['I03'].plot.imshow(vmin=0, vmax=100)

We've now put the data from two different instruments on the same gridded region. This will simplify any comparisons or future work we want to do with these data.

### Exercise:

**Time: 10 minutes**

Using the below two cells, plot the differences between the loaded ABI and VIIRS bands using xarray's plotting utilities (one difference plot per cell). Try using the `vmin` and `vmax` keyword arguments and see if limiting colorbar range brings out more information. Keep in mind that Xarray will try to choose the best colormap based on the data. You may want to force the colormap using `cmap='viridis'` or `cmap='RdBu_r'` or try [another colormap](https://matplotlib.org/tutorials/colors/colormaps.html) provided by matplotlib.

If time allows try plotting the average (mean) of the two bands.

In [ ]:
# C05 - I03


In [ ]:
# C07 - I04


<details>
    <summary>Solution (no cheating): C05 - I03</summary>

```
plt.figure()
diff = abi_max_scn['C05'] - viirs_max_scn['I03']
diff.plot.imshow(add_colorbar=True, vmin=-25, vmax=25, cmap='RdBu_r')
```

</details>




<details>
    <summary>Solution (no cheating): C07 - I04</summary>

```
plt.figure()
diff = abi_max_scn['C07'] - viirs_max_scn['I04']
diff.plot.imshow(add_colorbar=True, cmap='viridis')
```

</details>



<details>
    <summary>Solution: (C07 + I04) / 2</summary>

```
plt.figure()
diff = (abi_max_scn['C07'] + viirs_max_scn['I04']) / 2.0
diff.plot.imshow(add_colorbar=True, cmap='viridis')
```

</details>

By using resampling we are able to combine arrays that would not normally not be possible. This opens the doors to more types of analysis and the science that can be done.

## Cropping and Aggregating

Let's say we didn't want to change projection but only wanted to "cut out" a portion of the data or reduce the resolution of our data. Satpy provides the `Scene.crop` and `Scene.aggregate` methods to help us with this.

We'll start by using that same ABI Mesoscale Scene we used before. Let's say we wanted to cut out a specific region, but we only knew the longitude and latitude coordinates. By using information from the data's `AreaDefinition` we can use cut out a particular region of the data without having to do any expensive resampling.

In [ ]:
abi_scn = Scene(reader='abi_l1b', filenames=glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_meso/OR_ABI-L1b-RadM1-M3C??_G16_s20181312030*.nc'))
abi_scn.load(['C05'])

In [ ]:
crop_scn = abi_scn.crop(ll_bbox=[-102.0, 34.0, -100.0, 36.0])
crop_scn['C05'].shape

In [ ]:
plt.figure()
crop_scn['C05'].plot.imshow(cmap='viridis', vmin=0, vmax=100)

Perhaps we wanted to see the whole image, but we didn't need the highest resolution of all of the data. We can use the `Scene.aggregate` method to reduce our overall array size. Here we'll say we want to take the mean (default) of every 8x8 pixels in the `y` and `x` dimension.

In [ ]:
agg_scn = abi_scn.aggregate(y=10, x=10)
agg_scn['C05'].shape

In [ ]:
plt.figure()
agg_scn['C05'].plot.imshow(vmin=0, vmax=100)

The `aggregate` function takes a couple different possible functions for how it combines the data. Let's say instead of the `mean` we want the `max` value.

In [ ]:
plt.figure()
agg_scn = abi_scn.aggregate(y=10, x=10, func='max')
agg_scn['C05'].plot.imshow(vmin=0, vmax=100)

We've now finished learning about the `resample`, `crop`, and `aggregate` methods for taking data as provided and manipulating it to be the resolution, size, and region that we wish to analyze.

## Further Research

 * [Other resampling algorithms][1]
 * [Caching Resampling][2]
 * [Store areas for reuse][3]
 
  [1]: https://satpy.readthedocs.io/en/latest/api/satpy.html#module-satpy.resample
  [2]: https://satpy.readthedocs.io/en/latest/resample.html#caching-for-geostationary-data
  [3]: https://satpy.readthedocs.io/en/latest/resample.html#store-area-definitions